In [1]:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Source files will be saved in "/var/folders/pf/fm0v0g_52r754cfv3r4lqv980000gn/T/tmp0l2e27hz".


**Q1. Write a simple CUDA kernel that gives the sum of maximum element of 2 vectors and profile it for:**

- Execution on 1 thread and 1 block
- Execution on all threads of 1 block
- Execution on all threads of n blocks. Deciding n is upto you.

In [2]:
!touch max_vector.cu
"""
#include <stdio.h>
#include <cuda.h>
__global__ void maxElementKernel(float *vec, float *result, int n) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    extern __shared__ float sdata[];

    // Perform reduction to find the max element
    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (threadIdx.x < s) {
            sdata[threadIdx.x] = max(sdata[threadIdx.x], sdata[threadIdx.x + s]);
        }
        __syncthreads();
    }

    // Write result for this block to global mem
    if (threadIdx.x == 0) {
        result[blockIdx.x] = sdata[0];
    }
}

__global__ void sumOfMaxElements(float *vec1, float *vec2, float *result, int n) {
    __shared__ float max1;
    __shared__ float max2;

    // Find max of first vector
    maxElementKernel<<<1, blockDim.x, blockDim.x * sizeof(float)>>>(vec1, &max1, n);
    cudaDeviceSynchronize();

    // Find max of second vector
    maxElementKernel<<<1, blockDim.x, blockDim.x * sizeof(float)>>>(vec2, &max2, n);
    cudaDeviceSynchronize();

    // Sum the max elements and store the result
    if (threadIdx.x == 0) {
        result[0] = max1 + max2;
    }
}
"""

'\n#include <stdio.h>\n#include <cuda.h>\n__global__ void maxElementKernel(float *vec, float *result, int n) {\n    int idx = threadIdx.x + blockIdx.x * blockDim.x;\n    extern __shared__ float sdata[];\n\n    // Perform reduction to find the max element\n    for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {\n        if (threadIdx.x < s) {\n            sdata[threadIdx.x] = max(sdata[threadIdx.x], sdata[threadIdx.x + s]);\n        }\n        __syncthreads();\n    }\n\n    // Write result for this block to global mem\n    if (threadIdx.x == 0) {\n        result[blockIdx.x] = sdata[0];\n    }\n}\n\n__global__ void sumOfMaxElements(float *vec1, float *vec2, float *result, int n) {\n    __shared__ float max1;\n    __shared__ float max2;\n\n    // Find max of first vector\n    maxElementKernel<<<1, blockDim.x, blockDim.x * sizeof(float)>>>(vec1, &max1, n);\n    cudaDeviceSynchronize();\n\n    // Find max of second vector\n    maxElementKernel<<<1, blockDim.x, blockDim.x * sizeof(float)

In [ ]:
!nvcc -o max_vector max_vector.cu
!nvprof ./max_vector 1 1

In [ ]:
!nvcc -o max_vector max_vector.cu
!nvprof ./max_vector 1024 1

In [ ]:
!nvcc -o max_vector max_vector.cu
!nvprof ./max_vector 1024 2

**Q2.[OPTIONAL] Write a simple CUDA kernel that does matrix multiplication of 2 matrices.**